In [1]:
print("all ok")

all ok


In [2]:
corpus_of_documents = [
    "Take a leisurely walk in the park and enjoy the fresh air.",
    "Visit a local museum and discover something new.",
    "Attend a live music concert and feel the rhythm.",
    "Go for a hike and admire the natural scenery.",
    "Have a picnic with friends and share some laughs.",
    "Explore a new cuisine by dining at an ethnic restaurant.",
    "Take a yoga class and stretch your body and mind.",
    "Join a local sports league and enjoy some friendly competition.",
    "Attend a workshop or lecture on a topic you're interested in.",
    "Visit an amusement park and ride the roller coasters."
]

In [3]:
corpus_of_documents

['Take a leisurely walk in the park and enjoy the fresh air.',
 'Visit a local museum and discover something new.',
 'Attend a live music concert and feel the rhythm.',
 'Go for a hike and admire the natural scenery.',
 'Have a picnic with friends and share some laughs.',
 'Explore a new cuisine by dining at an ethnic restaurant.',
 'Take a yoga class and stretch your body and mind.',
 'Join a local sports league and enjoy some friendly competition.',
 "Attend a workshop or lecture on a topic you're interested in.",
 'Visit an amusement park and ride the roller coasters.']

In [4]:
user_query="I am an Indian and I am living in India"

In [5]:
documents="India is a country where Indians and other people can live"

In [6]:
from collections import Counter
import math

In [7]:
user_query_token=user_query.lower().split(" ")

In [8]:
user_query_token

['i', 'am', 'an', 'indian', 'and', 'i', 'am', 'living', 'in', 'india']

In [9]:
documents_token=documents.lower().split(" ")

In [10]:
documents_token

['india',
 'is',
 'a',
 'country',
 'where',
 'indians',
 'and',
 'other',
 'people',
 'can',
 'live']

In [11]:
query_counter=Counter(user_query_token)

In [12]:
query_counter

Counter({'i': 2,
         'am': 2,
         'an': 1,
         'indian': 1,
         'and': 1,
         'living': 1,
         'in': 1,
         'india': 1})

In [13]:
document_counter=Counter(documents_token)
document_counter

Counter({'india': 1,
         'is': 1,
         'a': 1,
         'country': 1,
         'where': 1,
         'indians': 1,
         'and': 1,
         'other': 1,
         'people': 1,
         'can': 1,
         'live': 1})

In [14]:
lst=[]
for i in query_counter.values():
    lst.append(i)

In [15]:
## senterce vector
lst

[2, 2, 1, 1, 1, 1, 1, 1]

In [16]:
mylist=[]
for token in query_counter.keys() & document_counter.keys():
    print(token)

and
india


In [17]:
mylist=[]
for token in query_counter.keys() & document_counter.keys():
    mylist.append(query_counter[token] * document_counter[token])

In [18]:
mylist

[1, 1]

In [19]:
dot_product=sum(mylist)

In [20]:
query_magnitude=math.sqrt(sum(query_counter[token] **2 for token in user_query_token))
query_magnitude

4.69041575982343

In [21]:
document_magnitude=math.sqrt(sum(document_counter[token] ** 2 for token in documents_token))
document_magnitude

3.3166247903554

In [22]:
similarity=dot_product/(query_magnitude * document_magnitude)
similarity

0.12856486930664499

In [23]:
def cosine_similarity(query,documents):
    ########spliting text#########
    query_token=query.lower().split()
    documents_token=documents.lower().split()

    ###### token counter##########
    query_count=Counter(query_token)
    document_count=Counter(documents_token)

    ########calculating dot product######
    common_list=[]
    for token in query_count.keys() & document_count.keys():
        common_list.append(query_count[token] * document_count[token])
    dot_product=sum(common_list)

    #######calculating magnitude#########
    query_magnitude=math.sqrt(sum(query_count[token] **2 for token in query_count.keys()))
    document_magnitude=math.sqrt(sum(document_count[token] **2 for token in document_count.keys()))

    ####### similarity score###########
    similarity_score=dot_product/(query_magnitude * document_magnitude) if query_magnitude * document_magnitude !=0 else 0

    return similarity_score



In [24]:
similarity=cosine_similarity(user_query,documents)
similarity

0.16116459280507606

In [25]:
query="Is yoga good for health"
document="Yoga is good for healthy living"
similarity=cosine_similarity(query,document)
similarity

0.7302967433402214

In [26]:
def return_response(query,corpus):
    similarities=[]
    for doc in corpus:
        similarity_score=cosine_similarity(query,doc)
        similarities.append(similarity_score)

    return corpus[similarities.index(max(similarities))]

In [27]:
corpus_of_documents

['Take a leisurely walk in the park and enjoy the fresh air.',
 'Visit a local museum and discover something new.',
 'Attend a live music concert and feel the rhythm.',
 'Go for a hike and admire the natural scenery.',
 'Have a picnic with friends and share some laughs.',
 'Explore a new cuisine by dining at an ethnic restaurant.',
 'Take a yoga class and stretch your body and mind.',
 'Join a local sports league and enjoy some friendly competition.',
 "Attend a workshop or lecture on a topic you're interested in.",
 'Visit an amusement park and ride the roller coasters.']

In [28]:
query="I love to go with my friends"
similar_sentence=return_response(query,corpus_of_documents)
similar_sentence

'Have a picnic with friends and share some laughs.'

In [29]:
query="I love to do yoga"
similar_sentence=return_response(query,corpus_of_documents)
similar_sentence

'Take a yoga class and stretch your body and mind.'

In [30]:
import requests
import json


In [31]:
def ollama_llm_call(user_input,similar_response):
    final_response=[]

    prompt="""You are an assistant. your job is to provide a short response on user query do not make long
    response with new information. You will be provided the recomended text context based on provided
    context generate your response.
    provided recomended context: {similar_response}.
    user query: {user_input}

    Compile a recommendation to the user based on the recommended activity and the user input."""

    url='http://localhost:11434/api/generate'
    data={
        "model":"llama2",
        "prompt": prompt.format(similar_response=similar_response,user_input=user_input)
    }
    headers={'Content-Type': 'application/json'}

    response=requests.post(url,data=json.dumps(data),headers=headers,stream=True)

    try:
        for line in response.iter_lines():
            if line:
                decoded_line=json.loads(line.decode('utf-8'))
                final_response.append(decoded_line['response'])
    finally:
        response.close()
    return final_response

In [ ]:
response=ollama_llm_call(user_query,similar_sentence)
response